In [1]:
%pwd
%cd d:\\Murgi\\code\\memes2024\\meme-research-2024

d:\Murgi\code\memes2024\meme-research-2024


d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

In [3]:
meme_df = pd.read_parquet("D:\Murgi\code\memes2024\meme-research-2024\data\meme_entries.parquet")
meme_df

,id,template_name,path,phash
0,0-days-without-lenny-simpsons,0-days-without-lenny-simpsons,D:/Memes2024/0-days-without-Lenny-Simpsons/0-d...,0fe9b236e884fc38
1,0-days-without-Lenny-Simpsons_23,0-days-without-lenny-simpsons,D:/Memes2024/0-days-without-Lenny-Simpsons/0-d...,07e5ba2ee806fe30
2,0-days-without-Lenny-Simpsons_1,0-days-without-lenny-simpsons,D:/Memes2024/0-days-without-Lenny-Simpsons/0-d...,1fed3226e804fe38
3,0-days-without-Lenny-Simpsons_24,0-days-without-lenny-simpsons,D:/Memes2024/0-days-without-Lenny-Simpsons/0-d...,07e1b83ee886fa38
4,0-days-without-Lenny-Simpsons_10,0-days-without-lenny-simpsons,D:/Memes2024/0-days-without-Lenny-Simpsons/0-d...,17e5b22ee806fe30
...,...,...,...,...
124196,skeleton-computer-84,skeleton-computer,D:/Memes2024/Skeleton-Computer/skeleton-comput...,e1e68b1929253d3d
124197,nick-young-178,nick-young,D:/Memes2024/Nick-Young/nick-young-178.jpg,4bd5913e14da4e55
124198,success-kid-original-196,success-kid-original,D:/Memes2024/Success-Kid-Original/success-kid-...,1d1d301f32c3f2e3
124199,Wrong-Neighboorhood-Cats_23,wrong-neighboorhood-cats,D:/Memes2024/Wrong-Neighboorhood-Cats/Wrong-Ne...,a950535053abb3bb


In [4]:
phashes = meme_df["phash"].values
phashes

array(['0fe9b236e884fc38', '07e5ba2ee806fe30', '1fed3226e804fe38', ...,
       '1d1d301f32c3f2e3', 'a950535053abb3bb', '4f420398f03f7c7a'],
      dtype=object)

In [5]:
def hex_to_hash(hexstr, hash_size=8):

    l = []
    count = hash_size * (hash_size // 4)
    if len(hexstr) != count:
        emsg = 'Expected hex string size of {}.'
        raise ValueError(emsg.format(count))
    for i in range(count // 2):
        h = hexstr[i*2:i*2+2]
        v = int("0x" + h, 16)
        l.append([v & 2**i > 0 for i in range(8)])
    return np.array(l).flatten()#.astype(int)
    
def precompute_vectors(hashes, phases_path):
    pickle_file = phases_path + '.pickle'
    if os.path.isfile(pickle_file): 
        with open(pickle_file, 'rb') as fo:
            hashes = pickle.load(fo)
        print('[w] fetch precomputed vectors from ', pickle_file, 'new processed', len(hashes))
        return np.array(hashes)
    else:
        # hashes = np.array(list(hashes.values()))
        hashes2 = []
        for hex_hash in tqdm(hashes, total=len(hashes)):
            try:
                hashes2.append(hex_to_hash(hex_hash))
            except Exception as e:
                print(hex_hash)
                print(str(e))
        with open(pickle_file, 'wb') as fo:
            pickle.dump(hashes2, fo)
    return np.array(hashes2)

phash_vectors = precompute_vectors(phashes, 'all_imgflip_phash_vectors')

  0%|          | 0/124201 [00:00<?, ?it/s]

In [11]:
from sklearn.preprocessing import LabelEncoder

X = phash_vectors
le = LabelEncoder()
le.fit(meme_df['template_name'])
meme_df['template_id'] = le.transform(meme_df['template_name'])
y = meme_df['template_id'].tolist()
X.shape, len(y)


((124201, 64), 124201)

In [12]:
import optuna
from optuna.samplers import TPESampler
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/


In [13]:
import constants
import mlflow
import constants

mlflow.set_tracking_uri('https://dagshub.com/levente-murgas/meme-research-2024.mlflow')
# os.environ['MLFLOW_TRACKING_URI']=f"'https://dagshub.com/levente-murgas/meme-research-2024.mlflow'"

# # Recommended to define as environment variables
os.environ['MLFLOW_TRACKING_USERNAME'] = constants.MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = constants.MLFLOW_TRACKING_PASSWORD

In [9]:
def get_experiment_id(name):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
      exp_id = mlflow.create_experiment(name)
      return exp_id
    return exp.experiment_id

exp_id = get_experiment_id("rnn_cross_val")
print(exp_id)

7


In [23]:
from sklearn.metrics import matthews_corrcoef, make_scorer, cohen_kappa_score, f1_score

def objective(trial):
    # -- Tune estimator algorithm
    radius = trial.suggest_int("radius", 6, 32)
    weights = trial.suggest_categorical("weights", ['uniform', 'distance'])
    rnn = RadiusNeighborsClassifier(radius=radius, weights=weights, metric='manhattan', outlier_label=-1)
    
    scoring = {'matthews_corrcoef': make_scorer(matthews_corrcoef),
              'cohen_kappa_score': make_scorer(cohen_kappa_score),
              'f1_score': make_scorer(f1_score, average='weighted')}
    
    # -- Make a pipeline
    pipeline = make_pipeline(rnn)

    # -- Cross-validate the features reduced by dimensionality reduction methods
    kfold = StratifiedKFold(n_splits=5)
    scores = cross_validate(pipeline, X, y, scoring=scoring, cv=kfold, return_estimator=True)

    try:
        with mlflow.start_run(experiment_id = exp_id):
            for i in range(5):
                mlflow.log_metrics({
                    "f1weighted": scores['test_f1_score'][i],
                    "kappa": scores['test_cohen_kappa_score'][i],
                    "mcc": scores['test_matthews_corrcoef'][i]
                }, step=i)
                mlflow.sklearn.log_model(
                    sk_model=scores['estimator'][i], 
                    artifact_path=f"rnn_{i}",
                    registered_model_name="rnn"
                )
    except:
        print("Logging was not succesful")
        pass
        
    optimizing_scores = np.array(scores['test_f1_score'])
    score = optimizing_scores.mean()
    return score

sampler = TPESampler(seed=42) # create a seed for the sampler for reproducibility
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=10)

[I 2024-04-19 14:19:37,189] A new study created in memory with name: no-name-1d815d3a-ac42-47cb-8048-707062511340
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in tr

In [24]:
# -- Have a look at the best trial
print("Best trial out of 10 is:")
study.best_trial

Best trial out of 10 is:


FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.9434861793418193], datetime_start=datetime.datetime(2024, 4, 19, 15, 52, 23, 791025), datetime_complete=datetime.datetime(2024, 4, 19, 16, 10, 43, 672731), params={'radius': 10, 'weights': 'distance'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'radius': IntDistribution(high=32, log=False, low=6, step=1), 'weights': CategoricalDistribution(choices=('uniform', 'distance'))}, trial_id=5, value=None)

In [25]:
# -- Have a look at the best parameters for the tuned model
print("Best parameters after tuning using mean accuracy:")
study.best_params

Best parameters after tuning using mean accuracy:


{'radius': 10, 'weights': 'distance'}

## Test with best params from grid search

Best trial out of 10 is:<br><br>
FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.9434861793418193], datetime_start=datetime.datetime(2024, 4, 19, 15, 52, 23, 791025), datetime_complete=datetime.datetime(2024, 4, 19, 16, 10, 43, 672731), params={'radius': 10, 'weights': 'distance'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'radius': IntDistribution(high=32, log=False, low=6, step=1), 'weights': CategoricalDistribution(choices=('uniform', 'distance'))}, trial_id=5, value=None)
<br><br>
Best parameters after tuning using mean accuracy:
{'radius': 10, 'weights': 'distance'}

In [15]:
from sklearn.metrics import matthews_corrcoef, make_scorer, cohen_kappa_score, f1_score


exp_id = get_experiment_id("rnn_cross_val_final")


rnn = RadiusNeighborsClassifier(radius=10, weights='distance', metric='manhattan', outlier_label=-1, n_jobs=-1)

scoring = {'matthews_corrcoef': make_scorer(matthews_corrcoef),
            'cohen_kappa_score': make_scorer(cohen_kappa_score),
            'f1_score': make_scorer(f1_score, average='weighted')}

# -- Cross-validate the features reduced by dimensionality reduction methods
kfold = StratifiedKFold(n_splits=5)
scores = cross_validate(rnn, X, y, scoring=scoring, cv=kfold, return_estimator=True)

with mlflow.start_run(experiment_id = exp_id):
    for i in range(5):
        mlflow.log_metrics({
            "f1weighted": scores['test_f1_score'][i],
            "kappa": scores['test_cohen_kappa_score'][i],
            "mcc": scores['test_matthews_corrcoef'][i]
        }, step=i)
        mlflow.sklearn.log_model(
            sk_model=scores['estimator'][i], 
            artifact_path=f"rnn_{i}",
            registered_model_name="rnn"
        )


d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2024\meme-research-2024\phash\lib\site-packages\sklearn\neighbors\_classification.py:768: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  warnings.warn(
d:\Murgi\code\memes2

In [17]:
mean_f1 = np.mean(scores['test_f1_score'])
mean_kappa = np.mean(scores['test_cohen_kappa_score'])
mean_mcc = np.mean(scores['test_matthews_corrcoef'])

print(f"Mean F1: {mean_f1}")
print(f"Mean Kappa: {mean_kappa}")
print(f"Mean MCC: {mean_mcc}")

Mean F1: 0.9434861793418193
Mean Kappa: 0.9126492723860304
Mean MCC: 0.9159318491882565
